In [3]:
############################################################################################
#  Parte inicial como siempre lo hacemos
############################################################################################
import findspark
findspark.init()

import pyspark

from pyspark.sql import SparkSession

from pyspark.sql import functions as F

spark = SparkSession.builder.appName("ejemplo con json file").getOrCreate()

############################################################################################
#  Se lee el archivo y se muestra cómo queda
############################################################################################
df = spark.read.option("multiline","true").json("ejemplo.json")
df.printSchema()
df.show()

############################################################################################
#  Se requiere poner columnas por separado.
#  Si no conoce, puede investigar el explode
#  https://sparkbyexamples.com/pyspark/pyspark-explode-array-and-map-columns-to-rows/
############################################################################################
base =(df
    .select(F.explode('viajes'), 'identificador')
    .select(F.explode('col'), 'identificador')
    .withColumn('origen', F.col('col.origen') )
    .withColumn('destino', F.col('col.destino') )
    .withColumn('km', F.col('col.km') )
    .withColumn('precio', F.col('col.precio') )
    .drop('col')
    )

base.printSchema()
base.show()


root
 |-- identificador: long (nullable = true)
 |-- viajes: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- destino: string (nullable = true)
 |    |    |    |-- km: string (nullable = true)
 |    |    |    |-- origen: string (nullable = true)
 |    |    |    |-- precio: string (nullable = true)

+-------------+--------------------+
|identificador|              viajes|
+-------------+--------------------+
|            1|[[{2, 10, 1, 5}],...|
+-------------+--------------------+

root
 |-- identificador: long (nullable = true)
 |-- origen: string (nullable = true)
 |-- destino: string (nullable = true)
 |-- km: string (nullable = true)
 |-- precio: string (nullable = true)

+-------------+------+-------+---+------+
|identificador|origen|destino| km|precio|
+-------------+------+-------+---+------+
|            1|     1|      2| 10|     5|
|            1|     2|      3| 20|     1|
|          